<h1>Captone Project</h1>
This notebook will be used for the Data Science Capstone

In [1]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<h1> Clustering Toronto</h1>

In [9]:
import pandas as pd
import requests

# Get the data from Toronto Open Data
	
url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "4def3f65-2a65-4a4f-83c4-b2a4aed72d46"}
package = requests.get(url, params = params).json()

	
for idx, resource in enumerate(package["result"]["resources"]):
    if resource["datastore_active"]:
        url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/datastore_search"
        p = { "id": resource["id"] }
        data = requests.get(url, params = p).json()
        df = pd.DataFrame(data["result"]["records"])
        break


'{"type": "Polygon", "coordinates": [[[-79.41469317817781, 43.6739104164259], [-79.41484930122832, 43.6743388247927], [-79.4155279126094, 43.67606998537741], [-79.4157867581137, 43.6767302521386], [-79.4164385645546, 43.6783329020511], [-79.4165782543862, 43.6787785512344], [-79.4167583944682, 43.6792484522149], [-79.41698203537291, 43.6798247440047], [-79.4170902471251, 43.6801047004014], [-79.4174797570886, 43.681051830379], [-79.4176666295158, 43.6815699096212], [-79.4176976020857, 43.68163169892501], [-79.4178116560388, 43.6819252777403], [-79.4181213711598, 43.6827105222857], [-79.4182734938897, 43.6830961662335], [-79.418301440907, 43.683167503583], [-79.4184313411692, 43.6834966841996], [-79.4184870757371, 43.6836294842548], [-79.4185410795905, 43.6837582054538], [-79.4184092922561, 43.683786393574], [-79.4182955283344, 43.6834937157887], [-79.4154265333151, 43.684065660547], [-79.4135200432832, 43.6844456915062], [-79.4134087861596, 43.6844695957643], [-79.4134686959176, 43.684

In [35]:
# Convert the polygons to shapes and find the centroids to use as the lat/lons for foursquare
#!pip install geopandas
import geopandas as gp
from shapely.geometry import shape
import json

geom = [shape(json.loads(i)) for i in df['geometry']]
gf = gp.GeoDataFrame(df, geometry=geom)
centroids = gf['geometry'].centroid

gf['LATITUDE'] = centroids.y
gf['LONGITUDE'] = centroids.x

gf.head()


,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry,CLASSIFICATION,CLASSIFICATION_CODE
0,11481,2480141,26005521,None,096,096,Casa Loma (96),Casa Loma (96),None,None,-79.408007,43.681852,17545105,3.678385e+06,8214.176485,"POLYGON ((-79.41469 43.67391, -79.41485 43.674...",None,None
1,11482,2480140,26005520,None,095,095,Annex (95),Annex (95),None,None,-79.404000,43.671585,17545121,5.337192e+06,10513.883143,"POLYGON ((-79.39414 43.66872, -79.39588 43.668...",None,None
2,11483,2480139,26005519,None,109,109,Caledonia-Fairbank (109),Caledonia-Fairbank (109),None,None,-79.455212,43.688569,17545137,2.955857e+06,6849.911724,"POLYGON ((-79.46021 43.68156, -79.46044 43.681...",None,None
3,11484,2480064,26005444,None,064,064,Woodbine Corridor (64),Woodbine Corridor (64),None,None,-79.315408,43.676773,17545153,3.052518e+06,7512.966773,"POLYGON ((-79.31485 43.66674, -79.31660 43.666...",None,None
4,11485,2480063,26005443,None,103,103,Lawrence Park South (103),Lawrence Park South (103),None,None,-79.406038,43.717212,17545169,6.211341e+06,13530.370002,"POLYGON ((-79.41096 43.70408, -79.41165 43.703...",None,None


In [23]:
# Define all the details needed to access the foursquare API
CLIENT_ID = 'ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM' # your Foursquare ID
CLIENT_SECRET = 'EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


In [33]:
# Taken from the DataScience Capstone lab
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('{name} ({lat}, {lon})'.format(name=name, lat=lat, lon=lng))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        print(url)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
nearby_venues = getNearbyVenues(names=gf['AREA_NAME'], latitudes=gf['LATITUDE'], longitudes=gf['LONGITUDE'])
nearby_venues



Casa Loma (96) (43.68185225756156, -79.40800655301227)
https://api.foursquare.com/v2/venues/explore?&client_id=ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM&client_secret=EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY&v=20180605&ll=43.68185225756156,-79.40800655301227&radius=500&limit=100
Annex (95) (43.671585254319105, -79.40400005427405)
https://api.foursquare.com/v2/venues/explore?&client_id=ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM&client_secret=EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY&v=20180605&ll=43.671585254319105,-79.40400005427405&radius=500&limit=100
Caledonia-Fairbank (109) (43.688568747993685, -79.45521207858553)
https://api.foursquare.com/v2/venues/explore?&client_id=ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM&client_secret=EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY&v=20180605&ll=43.688568747993685,-79.45521207858553&radius=500&limit=100
Woodbine Corridor (64) (43.676773242551675, -79.31540751369216)
https://api.foursquare.com/v2/venues/expl

South Parkdale (85) (43.63669879142937, -79.4393376794352)
https://api.foursquare.com/v2/venues/explore?&client_id=ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM&client_secret=EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY&v=20180605&ll=43.63669879142937,-79.4393376794352&radius=500&limit=100
Scarborough Village (139) (43.738652265494416, -79.216812606468)
https://api.foursquare.com/v2/venues/explore?&client_id=ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM&client_secret=EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY&v=20180605&ll=43.738652265494416,-79.216812606468&radius=500&limit=100
Broadview North (57) (43.6888253980725, -79.35562930263336)
https://api.foursquare.com/v2/venues/explore?&client_id=ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM&client_secret=EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY&v=20180605&ll=43.6888253980725,-79.35562930263336&radius=500&limit=100
Guildwood (140) (43.74882943278726, -79.19505423467297)
https://api.foursquare.com/v2/venues/explor

Agincourt North (129) (43.80544059101527, -79.26671244163076)
https://api.foursquare.com/v2/venues/explore?&client_id=ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM&client_secret=EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY&v=20180605&ll=43.80544059101527,-79.26671244163076&radius=500&limit=100
Junction Area (90) (43.667889908374924, -79.4714399796211)
https://api.foursquare.com/v2/venues/explore?&client_id=ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM&client_secret=EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY&v=20180605&ll=43.667889908374924,-79.4714399796211&radius=500&limit=100
Mount Dennis (115) (43.6881440214319, -79.49998942653895)
https://api.foursquare.com/v2/venues/explore?&client_id=ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM&client_secret=EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY&v=20180605&ll=43.6881440214319,-79.49998942653895&radius=500&limit=100
Edenbridge-Humber Valley (9) (43.67088668602387, -79.52245768813955)
https://api.foursquare.com/v2/ven

Woodbine-Lumsden (60) (43.69410687745314, -79.3111636645474)
https://api.foursquare.com/v2/venues/explore?&client_id=ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM&client_secret=EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY&v=20180605&ll=43.69410687745314,-79.3111636645474&radius=500&limit=100
Birchcliffe-Cliffside (122) (43.69468335216778, -79.26508925945065)
https://api.foursquare.com/v2/venues/explore?&client_id=ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM&client_secret=EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY&v=20180605&ll=43.69468335216778,-79.26508925945065&radius=500&limit=100
Bendale (127) (43.760367887461975, -79.25740154395558)
https://api.foursquare.com/v2/venues/explore?&client_id=ZXHDSENLBYGMWWPL5SCEESAVS0C15EJ1LPF3URRWQWG0NQLM&client_secret=EZNUXPGOOWSY1COTFAJAKJFVKW4IMKQWNH0P4MAWVNTBYZFY&v=20180605&ll=43.760367887461975,-79.25740154395558&radius=500&limit=100
Regent Park (72) (43.659992166811534, -79.36050877383667)
https://api.foursquare.com/v2/venues/e

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Casa Loma (96),43.681852,-79.408007,Sir Winston Churchill Park,43.683732,-79.409881,Park
1,Casa Loma (96),43.681852,-79.408007,Casa Loma Stables,43.679395,-79.410905,Museum
2,Casa Loma (96),43.681852,-79.408007,Casa Loma,43.677934,-79.409521,Castle
3,Casa Loma (96),43.681852,-79.408007,Baldwin Steps,43.677707,-79.408209,Historic Site
4,Casa Loma (96),43.681852,-79.408007,Spadina Museum: Historic House & Gardens,43.678883,-79.408858,History Museum
...,...,...,...,...,...,...,...
1386,Long Branch (19),43.592362,-79.533345,RBC Royal Bank,43.593882,-79.535475,Bank
1387,Long Branch (19),43.592362,-79.533345,Tim Hortons,43.592881,-79.539087,Coffee Shop
1388,Long Branch (19),43.592362,-79.533345,"Jeff, Rose & Herb's No Frills",43.593357,-79.538747,Grocery Store
1389,Long Branch (19),43.592362,-79.533345,The Beer Store,43.594755,-79.533663,Beer Store


In [37]:
toronto_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# Group the venues by neighbourhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt North (129),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.000000,0.055556,0.0,0.0,0.0,0.000000,0.0
1,Agincourt South-Malvern West (128),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
2,Alderwood (20),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
3,Annex (95),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
4,Banbury-Don Mills (42),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.041667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Woburn (137),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
94,Woodbine Corridor (64),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
95,Woodbine-Lumsden (60),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.111111,0.000000,0.0,0.0,0.0,0.000000,0.0
96,Yonge-Eglinton (100),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02439,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0


In [41]:
# Print each neighborhood with the top five venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North (129)----
                venue  freq
0  Dim Sum Restaurant  0.06
1   Convenience Store  0.06
2      Discount Store  0.06
3  Chinese Restaurant  0.06
4      Clothing Store  0.06


----Agincourt South-Malvern West (128)----
                      venue  freq
0        Chinese Restaurant  0.29
1             Shopping Mall  0.10
2  Mediterranean Restaurant  0.05
3        Seafood Restaurant  0.05
4                      Café  0.05


----Alderwood (20)----
               venue  freq
0        Pizza Place  0.50
1        Coffee Shop  0.25
2  Convenience Store  0.25
3                Zoo  0.00
4         Nail Salon  0.00


----Annex (95)----
            venue  freq
0            Café  0.13
1     Coffee Shop  0.09
2  Sandwich Place  0.09
3             Pub  0.09
4     Social Club  0.04


----Banbury-Don Mills (42)----
         venue  freq
0  Pizza Place  0.08
1   Shoe Store  0.08
2  Coffee Shop  0.08
3          Spa  0.04
4         Bank  0.04


----Bathurst Manor (34)----
            

                  venue  freq
0      Asian Restaurant  0.25
1   American Restaurant  0.25
2  Fast Food Restaurant  0.25
3     Convenience Store  0.25
4           Music Venue  0.00


----Mount Olive-Silverstone-Jamestown (2)----
                       venue  freq
0        Fried Chicken Joint  0.33
1                       Park  0.33
2                Coffee Shop  0.33
3               Optical Shop  0.00
4  Middle Eastern Restaurant  0.00


----Mount Pleasant West (104)----
                venue  freq
0  Italian Restaurant  0.10
1         Coffee Shop  0.10
2                Café  0.07
3    Sushi Restaurant  0.06
4         Pizza Place  0.06


----New Toronto (18)----
                venue  freq
0         Coffee Shop  0.17
1   Indian Restaurant  0.17
2  Italian Restaurant  0.17
3         Supermarket  0.17
4                 Pub  0.17


----Niagara (82)----
         venue  freq
0     Bus Stop  0.08
1   Restaurant  0.08
2          Pub  0.04
3          Bar  0.04
4  Coffee Shop  0.04


----North St

In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North (129),Japanese Restaurant,Discount Store,Beer Store,Frozen Yogurt Shop,Bank,Bakery,Fried Chicken Joint,Sandwich Place,Chinese Restaurant,Liquor Store
1,Agincourt South-Malvern West (128),Chinese Restaurant,Shopping Mall,Malay Restaurant,Pool Hall,Restaurant,Bank,BBQ Joint,Noodle House,Asian Restaurant,Cantonese Restaurant
2,Alderwood (20),Pizza Place,Convenience Store,Coffee Shop,Egyptian Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
3,Annex (95),Café,Pub,Sandwich Place,Coffee Shop,Pizza Place,Burger Joint,Middle Eastern Restaurant,French Restaurant,Park,Pharmacy
4,Banbury-Don Mills (42),Pizza Place,Coffee Shop,Shoe Store,Dessert Shop,Movie Theater,Furniture / Home Store,Clothing Store,Pharmacy,Bubble Tea Shop,Italian Restaurant


In [46]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 2, 4, 0, 4, 0, 4, 4, 4], dtype=int32)

In [81]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = pd.DataFrame(gf[['AREA_NAME', 'LATITUDE', 'LONGITUDE']])
toronto_merged.rename(columns={'AREA_NAME':'Neighborhood'}, inplace=True)


# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
#toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged = toronto_merged.dropna(how='any',axis=0) 
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

#!pip install folium
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors

#create map
map_clusters = folium.Map(location=[43.691313, -79.596364], zoom_start=11)

#set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['LATITUDE'], toronto_merged['LONGITUDE'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
   label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
   folium.CircleMarker(
       [lat, lon],
       radius=5,
       popup=label,
       color=rainbow[cluster-1],
       fill=True,
       fill_color=rainbow[cluster-1],
       fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [84]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park South (103),Sporting Goods Shop,Spa,Gym / Fitness Center,Chinese Restaurant,Coffee Shop,Flower Shop,Dim Sum Restaurant,Elementary School,Flea Market,Dance Studio
6,Henry Farm (53),Japanese Restaurant,Hotel,Intersection,Grocery Store,Sandwich Place,Yoga Studio,Event Space,Elementary School,Ethiopian Restaurant,Falafel Restaurant
8,Kingsview Village-The Westway (6),Pizza Place,Bus Line,Restaurant,Coffee Shop,Grocery Store,Egyptian Restaurant,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant
9,Kennedy Park (124),Discount Store,Playground,Coffee Shop,Yoga Studio,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Egyptian Restaurant
10,Pleasant View (46),Convenience Store,Thrift / Vintage Store,Burger Joint,Fast Food Restaurant,Breakfast Spot,Restaurant,Japanese Restaurant,Shopping Mall,Bank,Bakery
11,Steeles (116),Chinese Restaurant,Pharmacy,BBQ Joint,Fast Food Restaurant,Korean Restaurant,Supermarket,Sushi Restaurant,Baseball Field,Shopping Mall,Pizza Place
14,Humber Summit (21),Home Service,Furniture / Home Store,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
16,Yonge-Eglinton (100),Coffee Shop,Fast Food Restaurant,Restaurant,Gym,Deli / Bodega,Buffet,Supermarket,Juice Bar,Breakfast Spot,Poutine Place
18,North St.James Town (74),Coffee Shop,Grocery Store,Pizza Place,Sandwich Place,Filipino Restaurant,Juice Bar,Steakhouse,Breakfast Spot,Library,Café
21,Don Valley Village (47),Pizza Place,Fruit & Vegetable Store,Beer Store,Bakery,Grocery Store,Sandwich Place,Caribbean Restaurant,Fried Chicken Joint,Baseball Field,Fast Food Restaurant


In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Milliken (130),Park,Sandwich Place,Yoga Studio,Egyptian Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
7,Downsview-Roding-CFB (26),Business Service,Park,Moving Target,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
13,Brookhaven-Amesbury (30),Park,Yoga Studio,Egyptian Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market
23,Westminster-Branson (35),Gym / Fitness Center,Park,Yoga Studio,Egyptian Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
25,West Hill (136),Park,Tea Room,Construction & Landscaping,Gym / Fitness Center,Yoga Studio,Falafel Restaurant,Elementary School,Ethiopian Restaurant,Event Space,Farmers Market
41,Glenfield-Jane Heights (25),Park,Tea Room,Pool,Moving Target,Yoga Studio,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
59,Edenbridge-Humber Valley (9),Fast Food Restaurant,Park,Yoga Studio,Egyptian Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Farmers Market
63,Hillcrest Village (48),Park,Residential Building (Apartment / Condo),Yoga Studio,Egyptian Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
73,Parkwoods-Donalda (45),Park,Food & Drink Shop,Yoga Studio,Egyptian Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant
86,Bendale (127),Park,Convenience Store,Greek Restaurant,Yoga Studio,Falafel Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Farmers Market
